In [1]:
import requests
import onetimepass as otp
from urllib.parse import urlparse, parse_qs
from kiteconnect import KiteConnect

def get_request_token(kite, credentials: dict) -> str:
    
    session = requests.Session()
    # Step 1: Get login URL
    response = session.get(kite.login_url())
    # Step 2: Login form
    login_payload = {
        "user_id": credentials["username"],
        "password": credentials["password"],
    }
    login_response = session.post("https://kite.zerodha.com/api/login", login_payload)
    # Step 3: TOTP 2FA
    totp_payload = {
        "user_id": credentials["username"],
        "request_id": login_response.json()["data"]["request_id"],
        "twofa_value": otp.get_totp(credentials["totp_key"]),
        "twofa_type": "totp",
        "skip_session": True,
    }
    totp_response = session.post("https://kite.zerodha.com/api/twofa", totp_payload)
    print(totp_response)
    # Step 4: Extract request token
    try:
        response = session.get(kite.login_url())
        parse_result = urlparse(response.url)
        query_parms = parse_qs(parse_result.query)
    except Exception as e:
        import re
        pattern = r"request_token=[A-Za-z0-9]+"
        query_parms = parse_qs(re.findall(pattern, str(e))[0])
    request_token = query_parms["request_token"][0]
    return request_token


In [2]:
credentials={"username":"KMX177", 
             "password" : "Jack298!",
            "api_key":"wu80p2aelj2d73v5",
             "api_secret": "xorznn9fcocx1xww4uflrqprsorrij4t",
            "totp_key": "N6RGEW7E5VBDTGBLFOONFU3KOQZGR27G"}

In [3]:
kite = KiteConnect(api_key=credentials["api_key"])

In [4]:
req = get_request_token(kite, credentials)

<Response [200]>


In [5]:
request_token = req

In [6]:
data = kite.generate_session(request_token, api_secret=credentials["api_secret"])
access_token = data["access_token"]


In [7]:
access_token

'BPdyNZWhThEfBxtlWifCWME5rohwYha4'